Spatial Data Science

Homework 3

Mariam Hovhannisyan

1. Go to the Airbnb listings data, download the data for 2 cities of your choice, then load the OpenStreetMap city polygons for those cities with osmnx, subdivide them into grid cells of size 500-1000m, compute the spreading index profiles for both cities and plot them on the same plot.

In [1]:
import esda
import libpysal as lps
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd, networkx as nx
import osmnx as ox
from descartes import PolygonPatch
from shapely.geometry import Point, LineString, Polygon, MultiPolygon
import shapely
from pyproj import CRS
from tqdm import tqdm_notebook
from itertools import combinations, combinations_with_replacement

In [2]:
city1 = ox.geocode_to_gdf("Berlin, Germany")
city2 = ox.geocode_to_gdf("Munich, Germany")
colnames = ['id', 'name', 'host_id', 'neighbourhood', 'latitude', 'longitude']

city_gdf1 = pd.read_csv("Berlin_airbnb.csv", header=0,usecols = colnames)
city_gdf2 = pd.read_csv("Munich_airbnb.csv", header=0,usecols = colnames)

def city_gdf(city, city_gdf):
    city = city.to_crs("EPSG:3068")  #crs widely used for Germany
    geometry = city['geometry'].iloc[0]

    if isinstance(geometry, Polygon):
        geometry = MultiPolygon([geometry])

    #dividing city map into 900m square grids
    geometry_cut = ox.utils_geo._quadrat_cut_geometry(geometry, quadrat_width=1000)

    city['coords'] = city['geometry'].apply(lambda x: x.representative_point().coords[:])
    city['coords'] = [coords[0] for coords in city['coords']]

    polylist = [p for p in geometry_cut]

    west, south, east, north = city.unary_union.bounds
    polyframe = gpd.GeoDataFrame(geometry=polylist, crs = city.crs)
    
    geometry = [Point(xy) for xy in zip(city_gdf.latitude, city_gdf.longitude)] 
    crs = CRS.from_epsg(3068)
    city_gdf = gpd.GeoDataFrame(city_gdf, crs = crs, geometry=geometry)
    city_gdf.geometry= city_gdf.geometry.to_crs(polyframe.crs)
    percell = gpd.sjoin(polyframe, city_gdf, op='contains')
    homecounts = percell.groupby(percell.index)['index_right'].count()
    counts = pd.DataFrame(homecounts)
    counts.columns = ['counts']

    polycounts = polyframe.copy()
    polycounts['counts'] = counts.counts
    polycounts['counts'].fillna(0, inplace=True)
    polycounts['counts'] = polycounts['counts'].astype(int)
    
    return polycounts.tail()

In [3]:
brln_polycounts = city_gdf(city1, city_gdf1)
#brln_polycounts
mnch_polycounts = city_gdf(city2, city_gdf2)

In [4]:
def eucl(polycounts):
    n= len(polycounts)
    mat = np.zeros(shape=(n, n))
    for pair in tqdm_notebook(combinations(np.arange(n), 2)):
        mat[pair[0], pair[1]] = polycounts.geometry[pair[0]].centroid.distance(polycounts.geometry[pair[1]].centroid)

    eucl = np.maximum(mat, mat.transpose())
    return eucl

brln_eucl = eucl(brln_polycounts)
mnch_eucl = eucl(mnch_polycounts)
brln_eucl

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


0it [00:00, ?it/s]

KeyError: 0

In [ ]:
lval_brln = np.linspace(0, max(brln_polycounts.counts), 100) 
lval_mnch = np.linspace(0, max(mnch_polycounts.counts), 100)

cities_dist = {'Berlin data': [brln_polycounts.counts, brln_eucl, lval_brln], 
               'Munich data': [mnch_polycounts.counts, mnch_eucl, lval_mnch]}
shuffled_etas = {}  #dictionary with 2 key-value pair

for key, distribution in tqdm_notebook(cities_dist.items()):
    if key == 'Berlin data':
        eucl = brln_eucl 
        lval =lval_brln
    else:
        eucl = mnch_eucl
        lval = lval_mnch

    lorentz_vals = distribution.to_dict()
    etas = []

    s = [(k, lorentz_vals[k]) for k in sorted(lorentz_vals, key=lorentz_vals.get)]
    keys = []
    vals = []
    for k,v in s:
        keys.append(k)
        vals.append(v)

    vals = np.array(vals)
    keys = np.array(keys)
    L = np.cumsum(vals)/np.sum(vals)

    for i in lval:
        loubar_keys = keys[vals>=i]
        dist_mat = eucl[keys.reshape(-1,1), keys]  #will get matrix for Brln for the 2st loop, and for mnch for teh 2nd
        dist_corr = dist_mat[dist_mat>0]

        loubar_dist_mat = eucl[loubar_keys.reshape(-1,1), loubar_keys]
        loubar_dist = loubar_dist_mat.sum()

        loubar_dist_corr = loubar_dist_mat[loubar_dist_mat>0]
        eta = loubar_dist_corr.mean()/dist_corr.mean()
        etas.append(eta)
    etas = np.array(etas)
    etas = np.where(np.isnan(etas), 0, etas)
    lval = lval/lval.max()
    shuffled_etas[key] = [etas, lval]
    

fig, ax = plt.subplots(figsize=(16, 8))
for key, vals in shuffled_etas.items(): 
    ax.plot(vals[0], vals[1], linestyle='--', marker='o', markersize=4, linewidth=1,label=key) 

plt.xlabel(" ")
plt.ylabel("eta")
plt.ylim(0,1.5)
plt.show()

2. From the Airbnb listings data, get the neighbourhood files for one of the cities chosen above and conduct the spatial autocorrelation analysis you learned in this class for that city.

In [ ]:
brln_nbr = gpd.read_file("neighbourhoods.geojson", driver="GeoJSON")
crs = CRS.from_epsg(3068)
brln_nbr = brln_nbr.to_crs(crs)

#brln_nbr.type
brln_nbr.head()
brln_nbr.geometry.plot()

In [ ]:
city_gdf1 = pd.read_csv("Berlin_airbnb.csv", header=0,usecols = colnames)
geometry = [Point(xy) for xy in zip(city_gdf1.latitude, city_gdf1.longitude)] 
crs = CRS.from_epsg(3068)
brln_gdf = gpd.GeoDataFrame(city_gdf1, crs = crs, geometry=geometry)
brln_gdf.geometry= brln_gdf.geometry.to_crs(brln_nbr.crs)
brln_gdf

In [ ]:
brln_join = gpd.sjoin(brln_gdf,brln_nbr, op='contains')
brln_join.head()

counts = brln_join.groupby(brln_join.index)['index_right'].count()
counts = pd.DataFrame(counts)
counts.columns = ['counts']

plt.hist(counts.counts, bins=40)
plt.show()

In [ ]:
polycounts = brln_nbr.copy()
polycounts['counts'] = counts.counts
polycounts['counts'].fillna(0, inplace=True)
polycounts['counts'] = polycounts['counts'].astype(int)
polycounts

plt.rcParams.update({'font.size':16})
west, south, east, north = brln_nbr.unary_union.bounds

fig, ax = plt.subplots(figsize=(12,10), subplot_kw={'aspect':'equal'})

brln_nbr.plot(ax=ax, color='#000004')
polycounts.plot(column='counts', scheme='Quantiles', k=5, cmap='GnBu', legend=True, ax=ax)
ax.set_xlim(west, east)
ax.set_ylim(south, north)
ax.axis('off')
plt.show()

In [ ]:
wr = lps.weights.contiguity.Rook.from_dataframe(polycounts)
#print(wr.pct_nonzero)
s = pd.Series(wr.cardinalities)
s.plot.hist(bins=s.unique().shape[0]);

In [ ]:
wq = lps.weights.Queen.from_dataframe(polycounts)
wq.transform = 'r'

wq.neighbors

In [ ]:
#adding the closest neighbour to the disconnected ones
disconnected_neighs = polycounts.iloc[[83, 114, 120]]
disconnected_neighs

wk1 = lps.weights.distance.KNN.from_dataframe(polycounts, k=1)
wq.neighbors.append(83)
wq.neighbors.append(114)
wq.neighbors.append(120)

w_new = lps.weights.W(wq.neighbors)

In [ ]:
y = polycounts['counts']
ylag = lps.weights.lag_spatial(w_new, y)
polycounts['lag_counts'] = ylag

ylagq5 = mc.Quantiles(ylag, k=5)

In [ ]:
f, ax = plt.subplots(1,2,figsize=(20,8))
polycounts.plot(column='counts', ax=ax[0], edgecolor='grey', linewidth=0.25,
        scheme="quantiles",  k=5, cmap='GnBu')
ax[0].axis(polycounts.total_bounds[np.asarray([0,2,1,3])])
ax[0].set_title("Counts")
polycounts.plot(column='lag_counts', ax=ax[1], edgecolor='grey', linewidth=0.25,
        scheme='quantiles', cmap='GnBu', k=5)
ax[1].axis(polycounts.total_bounds[np.asarray([0,2,1,3])])
ax[1].set_title("Spatial Lag counts")
ax[0].axis('off')
ax[1].axis('off')
plt.show()

In [ ]:
y.median()
yb = y > y.median()
sum(yb)

labels = ["0 Low", "1 High"]
yb = [labels[i] for i in 1*yb] 
polycounts['yb'] = yb

fig, ax = plt.subplots(figsize=(12,10), subplot_kw={'aspect':'equal'})
polycounts.plot(column='yb', cmap='binary', edgecolor='grey', legend=True, ax=ax)

In [ ]:
sbn.kdeplot(sim_array, shade=True)
plt.vlines(jc.bb, 0, 0.075, color='r')
plt.vlines(jc.mean_bb, 0,0.075)
plt.xlabel('BB Counts')

In [ ]:
np.random.seed(10000)
mi = esda.moran.Moran(y, w_new)
#mi.I

sbn.kdeplot(mi.sim, shade=True)
plt.vlines(mi.I, 0, 1, color='r')
plt.vlines(mi.EI, 0,1)
plt.xlabel("Moran's I")

In [ ]:
w_new.transform = 'r'
lag_home_counts = lps.weights.lag_spatial(w_new, y)

b, a = np.polyfit(y, lag_home_counts, 1)
f, ax = plt.subplots(1, figsize=(9, 9))
plt.plot(y, lag_home_counts, '.', color='firebrick')
plt.vlines(y.mean(), lag_home_counts.min(), lag_home_counts.max(), linestyle='--')
plt.hlines(lag_home_counts.mean(), y.min(), y.max(), linestyle='--')

# red line of best fit using global I as slope
plt.plot(y, a + b*y, 'r')
plt.title('Moran Scatterplot')
plt.ylabel('Spatial Lag of Airbnb counts')
plt.xlabel('Airbnb counts')

In [ ]:
li = esda.moran.Moran_Local(y, w_new)
li.q
(li.p_sim < 0.05).sum()

sig = li.p_sim < 0.05
hotspot = sig * li.q==1
coldspot = sig * li.q==3
doughnut = sig * li.q==2
diamond = sig * li.q==4

spot_labels = [ '0 ns', '1 hot spot', '2 doughnut', '3 cold spot', '4 diamond']
labels = [spot_labels[i] for i in spots]

In [ ]:
from matplotlib import colors
hmap = colors.ListedColormap([ 'lightgrey', 'red', 'pink', 'lightblue', 'blue'])
f, ax = plt.subplots(1, figsize=(9, 9))
polycounts.assign(cl=labels).plot(column='cl', categorical=True, \
        k=2, cmap=hmap, linewidth=0.1, ax=ax, \
        edgecolor='white', legend=True)
ax.set_axis_off()
plt.show()